In [70]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch

In [71]:
from fastai.vision import *

In [72]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_boston
from torch import nn,optim
import torch as th
import syft as sy

In [73]:
hook = sy.TorchHook(th)
boston = load_boston()
boston_data = pd.DataFrame(boston.data)


In [74]:
print(boston.keys())


dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [75]:
bo = pd.DataFrame(boston.data)
bo.columns = boston.feature_names
print(bo.head())
boston_target = boston.target

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33  


In [76]:
print(boston_target)

[24.  21.6 34.7 33.4 ... 20.6 23.9 22.  11.9]


In [77]:
X=torch.tensor(boston_data.values,requires_grad=True,dtype=torch.float)
Y=torch.tensor(boston_target,requires_grad=True,dtype=torch.float)

In [78]:
print (X.shape)

torch.Size([506, 13])


In [79]:
Y=Y.reshape((506,1))

In [80]:
bob=sy.VirtualWorker(hook,id='bob')


In [81]:
bill=sy.VirtualWorker(hook,id='bill')
rupert = sy.VirtualWorker(hook,id='rupert')

In [82]:
bob_train= X[0:200].send(bob)
bob_target=Y[0:200].send(bob)

In [83]:
bill_train = X[200:400].send(bill)
bill_target=Y[200:400].send(bill)
test_data = X[400:]
test_target = Y[400:].send(rupert)

In [84]:
loss_function= nn.MSELoss()

In [85]:
loss_function_bob = nn.MSELoss()

In [86]:
datasets =[(bob_train,bob_target),(bill_train,bill_target)]

In [87]:
def federated(iterations=100):

    model=nn.Sequential(nn.Linear(13, 18), nn.ReLU(),
                       nn.Linear(18, 10), nn.ReLU(),
                       nn.Linear(10, 5), nn.ReLU(),
                       nn.Linear(5, 1))
    opt =optim.SGD(params=model.parameters(),lr=0.01)
    
    for iter in range (iterations):
   
        for _data,_target in datasets:
             # sending model
                model=model.send(_data.location)

             #training on the different machines
                opt.zero_grad()
                pred=model(_data)
                loss=loss_function(pred,_target)                                 
                loss.backward()
                opt.step()
             
             #Get trained model
                model=model.get()

                print(loss.get())


In [88]:
federated()

tensor(787.5753, requires_grad=True)
tensor(385.2756, requires_grad=True)
tensor(600.8577, requires_grad=True)
tensor(722.3762, requires_grad=True)
tensor(557.1412, requires_grad=True)
tensor(676.3432, requires_grad=True)
tensor(516.8979, requires_grad=True)
tensor(633.8062, requires_grad=True)
tensor(479.8549, requires_grad=True)
tensor(594.4968, requires_grad=True)
tensor(445.7610, requires_grad=True)
tensor(558.1672, requires_grad=True)
tensor(414.3841, requires_grad=True)
tensor(524.5891, requires_grad=True)
tensor(385.5104, requires_grad=True)
tensor(493.5515, requires_grad=True)
tensor(358.9431, requires_grad=True)
tensor(464.8601, requires_grad=True)
tensor(334.5005, requires_grad=True)
tensor(438.3349, requires_grad=True)
tensor(312.0153, requires_grad=True)
tensor(413.8104, requires_grad=True)
tensor(291.3330, requires_grad=True)
tensor(391.1335, requires_grad=True)
tensor(272.3115, requires_grad=True)
tensor(370.1628, requires_grad=True)
tensor(254.8196, requires_grad=True)
t